In [1]:
import random

# Class representing the Tic-Tac-Toe game
class TicTacToe:
    def __init__(self):
        # Initialize the board with 9 empty spaces (3x3 grid)
        self.board = [" " for _ in range(9)]

    # Check if a player has won by verifying all possible winning combinations
    def is_winner(self, player):
        win_conditions = [
            (0, 1, 2),  # top row
            (3, 4, 5),  # middle row
            (6, 7, 8),  # bottom row
            (0, 3, 6),  # left column
            (1, 4, 7),  # middle column
            (2, 5, 8),  # right column
            (0, 4, 8),  # diagonal top-left to bottom-right
            (2, 4, 6)   # diagonal top-right to bottom-left
        ]
        # Check if the player's symbols match any of the win conditions
        for condition in win_conditions:
            if all(self.board[i] == player for i in condition):
                return True
        return False

    # Return a list of available positions on the board (i.e., empty spaces)
    def available_moves(self):
        return [i for i in range(9) if self.board[i] == " "]

    # Make a move at a given position if it's available; returns True if successful
    def make_move(self, position, player):
        if self.board[position] == " ":
            self.board[position] = player  # Place player's mark on the board
            return True
        return False  # Move invalid if the position is already taken

In [2]:

# Class representing the agent that plays using the Minimax algorithm
class MinimaxAgent:
    def __init__(self, game):
        self.game = game  # Reference to the game board

    # Minimax algorithm to evaluate the best possible move
    def minimax(self, board, depth, is_maximizing):
        # Base cases for the recursion: check if the game is over
        if self.game.is_winner("X"):
            return -1  # X wins, return negative score
        elif self.game.is_winner("O"):
            return 1  # O wins, return positive score
        elif len(self.game.available_moves()) == 0:
            return 0  # Tie, return neutral score

        # Maximizing player's turn (O)
        if is_maximizing:
            best_score = -float('inf')  # Initialize to the lowest possible score
            for move in self.game.available_moves():
                board[move] = "O"  # Simulate O making a move
                score = self.minimax(board, depth + 1, False)  # Recursive call
                board[move] = " "  # Undo the move
                best_score = max(score, best_score)  # Update the best score
            return best_score
        # Minimizing player's turn (X)
        else:
            best_score = float('inf')  # Initialize to the highest possible score
            for move in self.game.available_moves():
                board[move] = "X"  # Simulate X making a move
                score = self.minimax(board, depth + 1, True)  # Recursive call
                board[move] = " "  # Undo the move
                best_score = min(score, best_score)  # Update the best score
            return best_score

    # Get the best move based on the Minimax algorithm
    def get_move(self):
        best_move = None
        best_score = -float('inf')  # Start with the lowest score
        # Iterate over all available moves to find the best one
        for move in self.game.available_moves():
            self.game.board[move] = "O"  # Simulate O making a move
            score = self.minimax(self.game.board, 0, False)  # Evaluate using Minimax
            self.game.board[move] = " "  # Undo the move
            if score > best_score:
                best_score = score  # Update the best score
                best_move = move  # Store the best move
        return best_move  # Return the best move found


In [3]:

# Class representing the agent that plays using random moves
class RandomAgent:
    def __init__(self, game):
        self.game = game  # Reference to the game board

    # Get a random move from the available positions
    def get_move(self):
        return random.choice(self.game.available_moves())

# Function to let the user choose an agent
def choose_agent():
    print("Choose an agent to play against:")
    print("1. Minimax Agent (Hard)")
    print("2. Random Agent (Easy)")
    choice = input("Enter 1 or 2: ")
    if choice == "1":
        return "minimax"
    elif choice == "2":
        return "random"
    else:
        print("Invalid choice, defaulting to Random Agent.")
        return "random"

In [4]:
'''from semantic_segmentation import postion_played
from pyniryo2 import *
import pyniryo
from matplotlib import pyplot as plt
import cv2 as cv
import time

# robot_ip_address = "10.10.10.10"
robot_ip_address = "10.10.10.10"

# Connect to robot & calibrate
robot = NiryoRobot(robot_ip_address)
robot.arm.calibrate_auto()
from recorded_positions import pickup_coordinates,dropoff_coordinates'''


'from semantic_segmentation import postion_played\nfrom pyniryo2 import *\nimport pyniryo\nfrom matplotlib import pyplot as plt\nimport cv2 as cv\nimport time\n\n# robot_ip_address = "10.10.10.10"\nrobot_ip_address = "10.10.10.10"\n\n# Connect to robot & calibrate\nrobot = NiryoRobot(robot_ip_address)\nrobot.arm.calibrate_auto()\nfrom recorded_positions import pickup_coordinates,dropoff_coordinates'

In [5]:
def get_pickup_coordinates(p):
    return pickup_coordinates.get(p,'Invalid_position')
def get_dropoff_coordinates(p):
    return dropoff_coordinates.get(p,'Invalid_position')
def go_up(position_coordinates):
    coordinates=[0,0,0.03 ,0,0,0]
    result = [a + b for a, b in zip(position_coordinates, coordinates)]
    robot.arm.move_pose(result)

def go_down(position_coordinates):
    coordinates=[0,0,0.03 ,0,0,0]
    result = [a + b for a, b in zip(position_coordinates, coordinates)]
    robot.arm.move_pose(result)

In [6]:
def play_move(pick_up_list,agent_move):
    position_coordinates = get_pickup_coordinates(pick_up_list[0])
    robot.arm.move_pose(position_coordinates)
    robot.tool.grasp_with_tool()
    go_up(position_coordinates)

    position_coordinates = get_dropoff_coordinates(f'p{agent_move + 1}')
    robot.arm.move_pose(position_coordinates)
    go_down(position_coordinates)
    robot.arm.move_pose(position_coordinates)
    robot.tool.release_with_tool()

    pick_up_list.pop(0)



In [7]:
from motion import movement
from motion import clear_out
from postions_played import positions_played

# Main game loop
def play_game():
    game = TicTacToe()  # Initialize the game board
    agent_type = choose_agent()  # Ask user to choose the agent

    if agent_type == "minimax":
        agent = MinimaxAgent(game)  # Use Minimax agent
    else:
        agent = RandomAgent(game)  # Use Random agent

    global player_positions_stored
    player_positions_stored=[]    # storing the positions which player has played
    global robot_positions_stored
    robot_positions_stored=[]     # storing positions which robot has played
    global pick_up_list
    pick_up_list=['c1','c2','c3','c4','c5','c6,''c7','c8','c9','c10']
    global pickup_index
    pickup_index = 0

    # Game loop
    while True:
        
        # Player X's turn (user)
        players_input=positions_played()


        if len(players_input) > 0:
            position_input = players_input[-1]
        print('only',position_input)
        

        position = int(position_input[1]) - 1 
        print('only',position)
        #print(int(position_input[1]) - 1 )
         # Convert to 0-based index
        if not game.make_move(position, "X"):  # If the move is invalid
            print("Invalid move! Try again.")
            continue
        player_positions_stored.append(f'p{position_input[-1]})
        
        if game.is_winner("X"):
            print("You win!")
            print( 'player position list',player_positions_stored)
            clear_out(robot_positions_stored,player_positions_stored)
            
            break
        elif len(game.available_moves()) == 0:
            print("It's a tie!")
            clear_out(robot_positions_stored,player_positions_stored)
            break

        # Agent's turn (O)

        
        agent_move = agent.get_move()  # Get the move from the agent

        movement(agent_move+1,pickup_index=pickup_index)

        game.make_move(agent_move, "O") # Agent makes its move
        print(f"Agent plays at position: p{agent_move+1}")
        robot_positions_stored.append(f'p{agent_move+1}')
        pickup_index+=1


        if game.is_winner("O"):
            print("Agent wins!")
            clear_out(robot_positions_stored,player_positions_stored)
            break
        elif len(game.available_moves()) == 0:
            print("It's a tie!")
            clear_out(robot_positions_stored,player_positions_stored)
            break
    print(player_positions_stored)
    print(robot_positions_stored)


# Start the game
play_game()


Choose an agent to play against:
1. Minimax Agent (Hard)
2. Random Agent (Easy)
Positions with blue: ['p1']
only p1
only 0
Agent plays at position: p2
Positions with blue: []
Positions with blue: ['p1']
only p1
only 0
Invalid move! Try again.
Positions with blue: []
Positions with blue: ['p1']
only p1
only 0
Invalid move! Try again.
Positions with blue: ['p1', 'p5']
only p5
only 4
Agent plays at position: p8
Positions with blue: ['p1', 'p9']
only p9
only 8
You win!
player position list ['1', '5', '9']


RobotCommandException: Expected type: float.
Given: Invalid_position